In [21]:
import pandas as pd

In [23]:
data=pd.read_csv("data_final.csv")
data.head()

,size,bold,italic,text_len,num_prefix,is_larger_than_common_font,text_case,is_bulleted,label
0,11.0,0,0,95,0,0,mixed,0,H3
1,11.0,0,0,87,0,0,mixed,0,H3
2,11.0,0,0,19,0,0,lower,0,H3
3,11.0,0,0,73,0,0,mixed,0,H3
4,13.0,1,0,36,1,1,mixed,0,H1


In [25]:
from sklearn.preprocessing import LabelEncoder

le_case = LabelEncoder()
le_label = LabelEncoder()

In [27]:
data['text_case']=le_case.fit_transform(data['text_case'])
data['label']=le_label.fit_transform(data['label'])

In [29]:
data.head()

,size,bold,italic,text_len,num_prefix,is_larger_than_common_font,text_case,is_bulleted,label
0,11.0,0,0,95,0,0,1,0,2
1,11.0,0,0,87,0,0,1,0,2
2,11.0,0,0,19,0,0,0,0,2
3,11.0,0,0,73,0,0,1,0,2
4,13.0,1,0,36,1,1,1,0,0


In [31]:
data['label'].describe()

count    1512.000000
mean        2.785053
std         0.624421
min         0.000000
25%         3.000000
50%         3.000000
75%         3.000000
max         3.000000
Name: label, dtype: float64

In [33]:
le_case.classes_

array(['lower', 'mixed', 'sentence', 'upper'], dtype=object)

In [35]:
le_label.classes_

array(['H1', 'H2', 'H3', 'P'], dtype=object)

In [37]:
data['label'].value_counts()

label
3    1324
2      81
1      77
0      30
Name: count, dtype: int64

In [39]:
X = data.drop(columns=["label"])
y = data['label']

In [41]:
y

0       2
1       2
2       2
3       2
4       0
       ..
1507    3
1508    3
1509    3
1510    3
1511    3
Name: label, Length: 1512, dtype: int64

In [17]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)

In [43]:
X_resampled, y_resampled = smote.fit_resample(X, y)

In [45]:
X_resampled

,size,bold,italic,text_len,num_prefix,is_larger_than_common_font,text_case,is_bulleted
0,11.000000,0,0,95,0,0,1,0
1,11.000000,0,0,87,0,0,1,0
2,11.000000,0,0,19,0,0,0,0
3,11.000000,0,0,73,0,0,1,0
4,13.000000,1,0,36,1,1,1,0
...,...,...,...,...,...,...,...,...
5291,10.451679,1,0,26,0,0,1,0
5292,9.700000,1,0,2,1,0,1,0
5293,12.000000,1,0,7,0,1,2,0
5294,10.000000,1,0,50,0,0,1,0


In [47]:
y_resampled.value_counts()

label
2    1324
0    1324
1    1324
3    1324
Name: count, dtype: int64

In [49]:
from sklearn.preprocessing import MinMaxScaler

In [51]:
scaler = MinMaxScaler()

X_resampled['size'] = scaler.fit_transform(X_resampled[['size']])
X_resampled['text_len'] = scaler.fit_transform(X_resampled[['text_len']])

In [53]:
X_resampled

,size,bold,italic,text_len,num_prefix,is_larger_than_common_font,text_case,is_bulleted
0,0.192308,0,0,0.686131,0,0,1,0
1,0.192308,0,0,0.627737,0,0,1,0
2,0.192308,0,0,0.131387,0,0,0,0
3,0.192308,0,0,0.525547,0,0,1,0
4,0.269231,1,0,0.255474,1,1,1,0
...,...,...,...,...,...,...,...,...
5291,0.171218,1,0,0.182482,0,0,1,0
5292,0.142308,1,0,0.007299,1,0,1,0
5293,0.230769,1,0,0.043796,0,1,2,0
5294,0.153846,1,0,0.357664,0,0,1,0


In [55]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

In [57]:
from sklearn.tree import DecisionTreeClassifier

In [59]:
treeclf = DecisionTreeClassifier(criterion = 'gini', splitter = 'best', min_samples_split = 2, min_samples_leaf = 1)
treeclf = treeclf.fit(X_train, y_train)

In [61]:
y_pred = treeclf.predict(X_test)

In [63]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [65]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.9726415094339622

Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.96      0.96       265
           1       0.96      0.97      0.96       265
           2       0.98      0.98      0.98       265
           3       0.99      0.98      0.99       265

    accuracy                           0.97      1060
   macro avg       0.97      0.97      0.97      1060
weighted avg       0.97      0.97      0.97      1060


Confusion Matrix:
 [[254  11   0   0]
 [  7 256   1   1]
 [  2   1 261   1]
 [  0   0   5 260]]


In [67]:
from sklearn.ensemble import RandomForestClassifier

rndForstclf = RandomForestClassifier(n_estimators = 2, criterion = 'gini', max_depth = 5, max_features='sqrt', min_samples_split=2, min_samples_leaf=3, n_jobs=1)
rndForstclf = rndForstclf.fit(X_train, y_train)

y_pred = rndForstclf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.8603773584905661

Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.90      0.87       265
           1       0.89      0.72      0.80       265
           2       0.89      0.87      0.88       265
           3       0.83      0.95      0.89       265

    accuracy                           0.86      1060
   macro avg       0.86      0.86      0.86      1060
weighted avg       0.86      0.86      0.86      1060


Confusion Matrix:
 [[238  22   0   5]
 [ 46 191  17  11]
 [  0   1 230  34]
 [  0   1  11 253]]


In [69]:
from sklearn.ensemble import GradientBoostingClassifier

grdbstcf = GradientBoostingClassifier(
    loss='log_loss',
    learning_rate=0.1,
    min_samples_split=2,
    min_samples_leaf=1,
    max_depth=3,
    n_estimators=150,
    subsample=1.0,
    criterion='friedman_mse',
    random_state=42  
)

grdbstcf = grdbstcf.fit(X_train, y_train)

y_pred = grdbstcf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.9566037735849057

Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.94      0.94       265
           1       0.93      0.95      0.94       265
           2       0.97      0.97      0.97       265
           3       0.99      0.97      0.98       265

    accuracy                           0.96      1060
   macro avg       0.96      0.96      0.96      1060
weighted avg       0.96      0.96      0.96      1060


Confusion Matrix:
 [[248  17   0   0]
 [ 13 251   1   0]
 [  2   3 258   2]
 [  0   0   8 257]]


In [70]:
import pickle
with open("model.pkl", "wb") as file:
    pickle.dump(treeclf, file)

with open("scaler.pkl", "wb") as scaler_file:
    pickle.dump(scaler, scaler_file)

with open("le_case.pkl", "wb") as le_file:
    pickle.dump(le_case, le_file)

with open("le_label.pkl", "wb") as le_file:
    pickle.dump(le_label, le_file)